In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

In [ ]:
#Save Credentials
with open('SpotifyID.txt', 'r') as f:
    client_id = f.read().replace('\n','')
with open('SpotifySecret.txt', 'r') as f:
    client_secret = f.read().replace('\n','')


#setup spotipy
username= "225a5fp37icg43v4vhnztptfy"
#username = "cz3ea2nv7vk9l4hkkoiqnvvua"
scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri='https://developer.spotify.com/dashboard/applications/ec25c1dfa01e4712b0d4d234f8b51a32'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [ ]:
def search_playlist(sp,new_playlist):
    playListID =""
    playlists = sp.user_playlists(username)
    for playlist in playlists["items"]:
        if (playlist["name"]==new_playlist):
            playListID = playlist["id"]
    return playListID

In [ ]:
def create_playlist(playlist_name,sp,username):
    
    #See if playlist already exists
    #if exists, return playlist id
    #else create new one
    playListID = search_playlist(sp,playlist_name)
    
    if (playListID==""):
        sp.user_playlist_create(username, playlist_name,public = False)
        playListID = search_playlist(sp,playlist_name)

    return playListID
    

In [ ]:
def add_songs(playListID,songs,duration,sp,username):
    song_set = set()
    while duration>0:
        bag = songs.sample(weights = [int(x) if str(x).isdigit() else 0 for x in songs.popularity])
        if bag.iloc[0,-5] not in song_set:
            song_set.add(bag.iloc[0,-5])
            uri = bag.iloc[0,-4]
            song_length = int(bag.iloc[0,-3])
            sp.user_playlist_add_tracks(username, playlist_id = playListID, tracks=[uri], position=None)
            duration = duration-song_length
    

In [ ]:
playListID = create_playlist("roadtrip3",sp,username)
duration=30*60*1000
states =["South Dakota"]
count = len(states)
for state in states:
    songs = pd.read_csv("./final_datasets/"+ state +".csv")
    songs = songs[songs["uris"]!="error"]
    songs = songs[songs["popularity"]!="0"]
    add_songs(playListID,songs,duration/count,sp,username)